<a href="https://colab.research.google.com/github/andygma567/Portfolio-Project-3/blob/main/My_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

We use a TFRecord dataset of annotated images of people wearing facemasks taken from Roboflow. We use this dataset to train an EfficientDet model from the Tensorflow 2 Object Detection API via transfer learning. Please see the references at the end for additional information discussing how these models work. 

All of the work in this notebook follows from the set of tutorials referenced at the end. 

Finally after training a model on our custom dataset we use a second Colab notebook to visualize the inference of our trained model. 

## Notable Features

Two notable features of this work are in the validation data evaluations and the image preprocessing that we add to the model configuration in the Section: 'Set up'. 

### Sequential valdiation dataset evaluation

The TF2 OD API originally is written so that the validation data is evaluated in parallel with the training by running a script to do evaluation of the validation dataset with a separate processor. This approach uses more RAM and preprocessors than we have available. Therefore we modify the source code in the TF2 OD script for training models in order to run the validation dataset evaluation as a postprocess after the model training is done. Please see Subsection: 'Adjust the Core Code'. 

### A method for modifying model configs using the TF2 OD API utils

In the Section: 'Set up' we add in additional image preprocessing steps. Adding in additional preprocessing steps to a model config file seems to be challenging to other users as well (please see Stack Overflow posts in Subsection: 'Change some of the image preprocessing') and the author is not aware of TF2 OD tutorials that explain how to adjust the model configs at the level of the config objects. In this work we present a method to create new preprocessing steps at the level of the config object using the utils from the TF2 OD API itself. This approach is different many of the approaches to modifying model configs by reading / writing text files with a custom script that are popular in online tutorials. 






# Set Up

## Download my Dataset

My first goal is to find some data that I can train on.

All training data needs to be in a TFRecord format to be used by  mdels in Tensorflow. TFRecord is a special kind of binary format that Google chooses to use because of it's speed for reading / writing and how space efficient it is. 

Normally, if I were to annontate my own image data set I would need to do the following:

1.   Annontate my own images using a popular image annontation software such as LabelImg
2.   Convert the file format (.XML ) to a TFRecord format using one of scripts in the TF2 Object Detection Repo utils folder (Please see Appendix)

I have a file of images of people wearing masks already in my Google drive. I got this dataset from Roboflow (add link later) and it's already in a TFRecord format.

I want to copy it and unzip it so that I can use it in my notebook. 

In [ ]:
# The ! in front is how we run command line arguments in a notebook. Alternatively we can also use %%bash
# Because of the white space I gotta use quotes
!cp "/content/drive/MyDrive/Mask Wearing.v1-416x416-black-padding.tfrecord.zip" .
# If a file of the same name already exists then it will ask if you want to overwrite the file
!unzip "/content/Mask Wearing.v1-416x416-black-padding.tfrecord.zip"

## (Optional) A Comment about Label Maps

I read online that tensorflow uses protobufs for efficiency and protobufs can be saved in 2 kinds of files: .pb (the unreadable binary form for large files) or .pbtxt (the readable text version)

My image data needs lablel map too. This is a file that you can write by hand if you want (just save it as a .pbtxt). 

It seems like because I'm using a roboflow dataset the label map was already made for me. If I didn't have the labels in .pbtxt format there's the script that Tanner uses from the raccoon dataset that looks like it works for converting VOC XML to Tfrecord format. 

[Link to Tanner's tutorial blog](https://gilberttanner.com/blog/tensorflow-object-detection-with-tensorflow-2-creating-a-custom-model/)

Also I read the [source code for label_map_util](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/label_map_util.py)

In [ ]:
# My labels map is a .pbtxt file that looks like this

# It's important that the id starts at index 1 because index = 0 is reserved for 'no detection' 
'''
item {
    name: "mask",
    id: 1,
    display_name: "mask"
}
item {
    name: "no-mask",
    id: 2,
    display_name: "no-mask"
}
'''

## Installing TF2 Object Detection API 

Three of the important libraries that we use in this notebook are CUDA (the code that manages the GPU usage), Tensorflow, and the TF2 OD repo. Google Colab comes preinstalled with these CUDA and Tensorflow but it's important to check that the installed versions of these libraries are compatbile. 

I ran into [this compatability error](https://stackoverflow.com/questions/71000120/colab-0-unimplemented-dnn-library-is-not-found) when I first ran this notebook. 

I think updating CUDA to the latest version fixed my issue. Sometimes you need to do some trial and error work with different installations until you find a set that works for you. I'm using tf v 2.9, libcudnn8 8.5.0.96-1+cuda11.7, and the TF OD repo in August 2022. 

In [ ]:
import tensorflow as tf
print(tf.__version__)

'2.9.1'

In [ ]:
# Check libcudnn8 version - it's pretty up to date 8.0.5.39-1+cuda11.1
!apt-cache policy libcudnn8
# This installs the a more uptodate version of CUDA
!apt install --allow-change-held-packages libcudnn8=8.5.0.96-1+cuda11.7

## Get the Tensorflow Object Detection Repo

Tensorflow keeps a repo on Github called Model Garden (Insert link later). Model Garden contains all of their state of the art models and in the research folder is there collection of state of the art object detection models that we will use. 


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3467, done.
remote: Counting objects: 100% (3467/3467), done.
remote: Compressing objects: 100% (2904/2904), done.
remote: Total 3467 (delta 898), reused 1488 (delta 506), pack-reused 0
Receiving objects: 100% (3467/3467), 46.88 MiB | 21.53 MiB/s, done.
Resolving deltas: 100% (898/898), done.


## Adjust the Core Code

I modify the source code as described in this blog post:
https://thachngoctran.medium.com/make-tensorflows-object-detection-validation-a-true-post-process-c45785f08d3a

This will make evaluating the model on my validation data a post process that I can run right after the training instead of running the evaluation on validation data in paralell. References on how to run a script in parallel are listed in the appendices at the end. 

I changed the file /content/models/official/object_detection/model_lib_v2.py in the source code and saved it as seq_eval_model_lib_v2.py in my Google Drive. 

The following code gets my changed python code and uses it to overwrite the file model_lib_v2.py that originally came from the Github repo download. 

It is important that this step is done before compiling the protos and installing the Github repo. 

In [ ]:
# Get my modifed script for sequential evaluation of the validation data
# and replace the existing script that is written for parallel validation eval
!cp "/content/drive/MyDrive/Object Detection/seq_eval_model_lib_v2.py" .
!mv "seq_eval_model_lib_v2.py" "model_lib_v2.py"
!mv "model_lib_v2.py" "/content/models/research/object_detection/model_lib_v2.py"

### The changes made to model_lib_v2.py

Change 1)

In module "model_lib_v2.py", function train_loop(), line 452: Replace
```
...
checkpoint_max_to_keep=7,
...
```
with 
```
checkpoint_max_to_keep=A_LARGE_NUM,
```
I set the parameter max_checkpoint_to_keep to be None. According to this API and this source code setting it to be None means it will save every checkpoint: 

*   [source code on TF OD github](https://github.com/tensorflow/models/blob/master/research/object_detection/model_lib_v2.py)
*   [TF Checkpoint Manager API](https://www.tensorflow.org/api_docs/python/tf/train/CheckpointManager)

Change 2)

See model_lib_v2.py, function eval_continuously(), line 1136: Replace

```
for latest_checkpoint in tf.train.checkpoints_iterator(
    checkpoint_dir, timeout=timeout, min_interval_secs=wait_interval):
  ...
ckpt.restore(latest_checkpoint).expect_partial()
```
with 

```
import regex as re
import glob
def natural_sort(l): 
  convert = lambda text: int(text) if text.isdigit() else text.lower() 
  alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
  return sorted(l, key = alphanum_key)
for latest_checkpoint in natural_sort(list(set(map(lambda n: n[:n.index(".")], glob.glob(f"{checkpoint_dir}/ckpt-*.*"))))):
```




## Install the Object Detection Repo

This code installs all of the protos using a proto compiler. The compiler reads all of the .proto files and then saves new .py scripts in the current directory (as specified by the --python_out=. argument)

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import 
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

## I need to download the models and config files

By convention every blog tutorial says to download a model checkpoint as a tar.gz file with !wget

I think the tar.gz file is just the training checkpoint for loading the pretrained weights. I got the link by going to the TF2 model zoo and hovering my mouse over the model and then copying the link address by right clicking. I really don't know if there is another way to do this

Here is a link to the [TF2 OD Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [ ]:
%mkdir /content/models/my_model_dir/
%cd /content/models/my_model_dir/

/content/models/my_model_dir


In [ ]:
# I'm getting the efficientdet_d0 maybe next time it would be better to use efficientdet_d4
!wget {"http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz"}

# This tar command extracts the file
# I think -x tells them that we're extracting as opposed to -c for compressing
# and -f is to let the script know that the next argument is the name of the file
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz

# This extracted directory also comes with the config file and the checkpoints in it

--2022-08-26 08:34:38--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.5.128, 2a00:1450:400c:c1b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.5.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M  32.4MB/s    in 0.9s    

2022-08-26 08:34:40 (32.4 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]



## Write a config file

In [ ]:
# These are the paths to the data files
BASE_CONFIG_PATH = 'efficientdet_d0_coco17_tpu-32/pipeline.config'
TRAIN_RECORD_PATH = '/content/train/People.tfrecord'
VAL_RECORD_PATH = '/content/valid/People.tfrecord'
# When you load checkpoints you give it the prefix of the two files .data and .index
FINE_TUNE_CKPT_PATH = 'efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'
LABEL_MAP_PATH = '/content/train/People_label_map.pbtxt'
SAVE_MY_CONFIG_PATH = '/content/models/my_model_dir'

# Read the configuration file into a dictionary of config objects
configs = config_util.get_configs_from_pipeline_file(BASE_CONFIG_PATH)
label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
NUM_CLASSES = len(label_map.keys())
# I think .WhichOneOf() is for a config object and it returns a str that's describes the instance of the config obj
meta_arch = configs["model"].WhichOneof("model") # The model architecture

NUM_CLASSES = 2 # The mask dataset has 2 classes
BATCH_SIZE = 16 # A lot of tutorials were saying batch_size 32 uses too much RAM
NUM_STEPS = 400 # It takes about 1 sec/ step and 1000 steps takes roughly 16min
CKPT_EVERY_N = 200 

# Some tutorials use a parameter called num_eval_steps to do an evaluation after 
# a specified number of training steps but I believe that this is deprecated in TF2

## (Optional) Change some of the Image Preprocessing

We will now add some additional image preprocessing. When we compile the protos the compiler will create new python files. We will use the newly created file *preprocessor_pb2.py* to create new preprocessing steps. 

A full list of what preprocessing options are available can be found in either the *preprocessor_pb2.py* file or in the [preprocessor.proto file in the TF2 OD repo](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto)

I describe how to add new preprocessing configurations in more detail in my answer to these posts on Stack Overflow: 
* [Dynamically Editing Pipeline Config...](https://stackoverflow.com/questions/55323907/dynamically-editing-pipeline-config-for-tensorflow-object-detection)
* [How to change data augmentation parameters dynamically...](https://stackoverflow.com/questions/65768729/how-to-change-data-augmentation-parameters-dynamically-in-the-config-file-of-ten)

In [ ]:
from object_detection.protos import preprocessor_pb2

# Construct a new PreprocessingStep() object
my_data_aug_1 = preprocessor_pb2.PreprocessingStep()
# Set the PreprocessingStep object's field
# random_rgb_to_gray has only one subfiell 'probability' that can be set
my_data_aug_1.random_rgb_to_gray.probability = 0.1

# I don't think it's possible to set a PreprocessingStep() to have multiple 
# preprocessing fields. Things just get overwritten

my_data_aug_2 = preprocessor_pb2.PreprocessingStep()
my_data_aug_2.random_adjust_brightness.max_delta = 0.2

my_data_aug_3 = preprocessor_pb2.PreprocessingStep()
my_data_aug_3.random_adjust_contrast.min_delta = 0.8
my_data_aug_3.random_adjust_contrast.max_delta = 1.25

my_data_aug_4 = preprocessor_pb2.PreprocessingStep()
my_data_aug_4.random_adjust_hue.max_delta = 0.02

# configs['train_config'].data_augmentation_options is a list of PreprocessingStep objects 
configs['train_config'].data_augmentation_options.append(my_data_aug_1)
configs['train_config'].data_augmentation_options.append(my_data_aug_2)
configs['train_config'].data_augmentation_options.append(my_data_aug_3)
configs['train_config'].data_augmentation_options.append(my_data_aug_4)

print(configs['train_config'].data_augmentation_options)

[random_horizontal_flip {
}
, random_scale_crop_and_pad_to_square {
  output_size: 512
  scale_min: 0.10000000149011612
  scale_max: 2.0
}
, random_rgb_to_gray {
  probability: 0.10000000149011612
}
, random_adjust_brightness {
  max_delta: 0.20000000298023224
}
, random_adjust_contrast {
  min_delta: 0.800000011920929
  max_delta: 1.25
}
, random_adjust_hue {
  max_delta: 0.019999999552965164
}
]


We use the utils in the repo to build a config file using the steps explained in here: https://github.com/yasserius/tf2-object-detection-api



In [ ]:
# I think this is for updating / overwriting existing stuff in the config file
override_dict = {
  'model.{}.num_classes'.format(meta_arch): NUM_CLASSES,
  'train_config.batch_size': BATCH_SIZE,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VAL_RECORD_PATH,
  'train_config.fine_tune_checkpoint': FINE_TUNE_CKPT_PATH,
  'label_map_path': LABEL_MAP_PATH, 
  'train_config.use_bfloat16': False, # Set this to True only if you use TPUs
  'train_config.num_steps': NUM_STEPS,
  'train_config.fine_tune_checkpoint_type': "detection", # b/c we are doing detection not classification
}
# Alternatively we could have modified the subfields of elements in the configs dictionary
# e.g. configs['train_config'].train_input_path = TRAIN_RECORD_PATH (I think)

# The Hparams arg is deprecated in TF2 so we use kwargs_dict to override the existing configs
configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict) 
# Make a new pipeline config proto
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
# This method can only save a config as 'pipeline.config'
config_util.save_pipeline_config(pipeline_config, SAVE_MY_CONFIG_PATH)

In [ ]:
!mv pipeline.config my_pipeline.config
%cat my_pipeline.config 

# Train

We can use Tensorboard to monitor the training of our model. 

First we use enable our internet browser to allow certain third party cookies so that we can see the Tensorboard in the Colab notebook. In my case because I use Google Chrome's browser I changed the privacy settings to allow 3rd party cookies from [*.]colab.research.google.com

For more information on how to enable cookies for just colab please see this Stack Overflow post [Allow a Google Colab domain cookies on chrome](https://stackoverflow.com/questions/50289535/allow-a-google-colab-domain-cookies-on-chrome)

In order to monitor training with Tensorboard it is important to run a cell to open Tensorboard before running the cell with the training script. We can refresh the Tensorboard while the training script is running in order to monitor our progress. 

In [ ]:
# Make a training directory that will hold all of our training and validation evaluation 
# TFevent files and our checkpoint files from training
!mkdir training

In [ ]:
# Set the file path to our training directory 
MODEL_DIR = 'training'
# Set the file path to our previously made pipeline config file
MY_CONFIG_PATH = '/content/models/my_model_dir/my_pipeline.config'

In [ ]:
# I need to keep clicking refresh as it continues to train in order to see the graphs in tensorboard
%load_ext tensorboard
%tensorboard --logdir 'training'

In [ ]:
# The parameter --alsologtostderr is to save the logs / checkpoints into the model_dir directory
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={MY_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr \
    --checkpoint_every_n={CKPT_EVERY_N} \

# Run Evaluation on Validation Data

In [ ]:
# The parameter sample_1_of_n_eval_examples=1 takes every single image from an eval batch for evaluation
# no image is skipped
!python  /content/models/research/object_detection/model_main_tf2.py \
    --alsologtostderr \
    --pipeline_config_path="my_pipeline.config" \
    --model_dir="training" \
    --sample_1_of_n_eval_examples=1 \
    --checkpoint_dir="training"

# Run the Model on the Test Data

We use the Tensorboard to hand-select a checkpoint to restor our model from and then we run the model on the test data set. We will rewrite the file 'checkpoint' in the training directory because the models run based on the latest checkpoint listed in this text file. 

We create a new directory for testing and copy the relevant checkpoints to that directory for the *model_main_tf2.py* script

In [ ]:
# Choose my checkpoint for testing the model from 
MY_CKPT = "ckpt-17"

In [ ]:
# Rewrite the checkpoint file according to my choosen checkpoint
# This code rewrites the latest checkpoint in the 'checkpoint' file to be MY_CKPT
with open(os.path.join(MODEL_DIR, 'checkpoint'), mode = 'r') as ckpt_file:
  lines = ckpt_file.readlines()
  first_line = lines[0].split('\"')
  first_line[1] = MY_CKPT
  lines[0] = '\"'.join(first_line)
with open(os.path.join(MODEL_DIR, 'checkpoint'), mode = 'w') as ckpt_file:
  ckpt_file.write(''.join(lines))

In [ ]:
# Create a directory and copy over all of the checkpoint files that I need to run an evaluation
import glob

!rm -r testing
!mkdir "testing"

for file_name in glob.glob(os.path.join(MODEL_DIR, MY_CKPT) + '*'):
  print(file_name)
  !cp {file_name} 'testing'
!cp '/content/models/my_model_dir/training/checkpoint' 'testing'

training/ckpt-17.index
training/ckpt-17.data-00000-of-00001


In [ ]:
# Write a custom config file using the test data as the eval input
import re
from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

TEST_RECORD_PATH = '/content/test/People.tfrecord'
SAVE_TEST_CONFIG_PATH = '/content/models/my_model_dir/testing'

# We make a new config file for the testing
configs = config_util.get_configs_from_pipeline_file(MY_CONFIG_PATH)
override_dict = {
  'eval_input_path': TEST_RECORD_PATH
}
configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict) 
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, SAVE_TEST_CONFIG_PATH)

In [ ]:
!mv testing/pipeline.config testing/test_pipeline.config
%cat testing/test_pipeline.config 

In [ ]:
# Open a new Tensorboard to monitor the results of our model on the testing data
%tensorboard --logdir 'testing'

In [ ]:
!python  /content/models/research/object_detection/model_main_tf2.py \
    --alsologtostderr \
    --pipeline_config_path="/content/models/my_model_dir/testing/test_pipeline.config" \
    --model_dir="testing" \
    --sample_1_of_n_eval_examples=1 \
    --checkpoint_dir="testing"

# Save and Export the Model

We save the model using the *exporter_main_v2.py* script. This script will save a 'SavedModel' directory in another directory it creates from the 
'output_directory' parameter. 

Then we copy this SavedModel directory to my Google Drive. 

In [ ]:
OUTPUT_DIR = 'trained_model'
LAST_MODEL_PATH = 'training'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --trained_checkpoint_dir={LAST_MODEL_PATH} \
    --output_directory={OUTPUT_DIR} \
    --pipeline_config_path={MY_CONFIG_PATH}

In [ ]:
# Save the trained model to my Google Drive and we'll test the model in another Colab
# The flag -r is for 'recursive' in order to copy the entire directory over 
!cp -r /content/models/my_model_dir/trained_model /content/drive/MyDrive

# Conclusions / Further Work

In another Colab we will test the inference abilities of our trained model. 

# Appendix A (Running Validation eval in Parallel)

Originally the validation is code is meant to be executing the same model_main_tf2.py script (while passing in an arugment for checkpoint_directory) in another terminal window while the first execution of model_main_tf2.py is still running. The evaluation is done by waiting for new checkpoints / tf events to be produced by the first (training) script. I think the default waiting time can be changed by passing in an argument to the model_main_tf2.py (for evaluation) script call. 

When I try to run the training and evaluation in parallel (like it was designed) it crashes because of I run out of RAM. I think I would need about 16GB of RAM and at least two processors to do this. 

People suggest running a parallel script by opening another terminal (and even allocating a processor e.g. CPU if needed). Please see these posts for reference of parallel processing:
* https://stackoverflow.com/questions/64510791/tf2-object-detection-api-model-main-tf2-py-validation-loss
* https://github.com/yasserius/tf2-object-detection-api
* https://techzizou.com/training-an-ssd-model-for-a-custom-object-using-tensorflow-2-x/#tf2_step14



# Appendix B (Converting Pascal VOC .xml format to TFRecord format)

Here are a couple of links related how to convert files to TFRecord format using the utils in the TF2 OD repo. 

*   [TF2 OD Dataset_tools](https://github.com/tensorflow/models/tree/master/research/object_detection/dataset_tools)
*   [Convert Pascal VOC dataset into TFRecord format in 12 Steps.](https://medium.com/@shwetaka1988/convert-pascal-voc-dataset-into-tfrecord-format-in-12-steps-eedbfa37dac1)
*   [Understanding TFRecord format](https://www.kaggle.com/code/gauravchopracg/understanding-tfrecord-format/notebook)
*   [Keras Tutorial - Creating TFRecords](https://keras.io/examples/keras_recipes/creating_tfrecords/#generate-data-in-the-tfrecord-format)
*   [Roboflow App method to create TFRecords](https://blog.roboflow.com/create-tfrecord/)

# Appendix C (Alternative ways to do Object Detection)

At the time of writing this notebook (September 2022) I found at least two other popular ways to train custom object detection models. 

## Option 1) Tensorflow Lite model maker 
This is an experimental module in Tensorflow Lite. This approach can create and train a tensorflow lite model on a custom dataset in just few lines of code. The notation is very similar to what is used to train cars models making it very easy to pick up. However, the model options are very limited to just one kind of object detection model and there's much less customization allowed. For example, you can't have new image pre-processing to your model. 



*   [12 min youtube tutorial](https://www.youtube.com/watch?v=-ZyFYniGUsw)
> (Also checkout the colab notebook in the video description)
*   [Object Detection with TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker/object_detection)


## Option 2) Detectron2

This is another popular object detection repo built and maintained by Meta (formerly facebook). It offers similar services to the TF2 Object Dectection API but it is build for PyTorch. It is notable that Detectron2 offers a many more models for creating masks and doing image segmentation than the TF2 OD API.
* [Detectron2 Model Zoo ](https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md)

# References 

**Tutorials**

1.   https://neptune.ai/blog/how-to-train-your-own-object-detector-using-tensorflow-object-detection-api
2.   https://neptune.ai/blog/tensorflow-object-detection-api-best-practices-to-training-evaluation-deployment
3.   https://gilberttanner.com/blog/tensorflow-object-detection-with-tensorflow-2-creating-a-custom-model/
4.   https://github.com/yasserius/tf2-object-detection-api
5.   https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_saved_model.html
6.   https://blog.roboflow.com/train-a-tensorflow2-object-detection-model/
7.   [Roboflow Public Datasets](https://public.roboflow.com/)
8.   https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/eager_few_shot_od_training_tf2_colab.ipynb

**EfficientNet**

1.   [EfficientNet: Improving Accuracy and Efficiency through AutoML and Model Scaling](https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html)
2.   [A Medium Blogpost on EfficientNet](https://medium.com/@nainaakash012/efficientnet-rethinking-model-scaling-for-convolutional-neural-networks-92941c5bfb95)

**EfficientDet**

1.   [A nice series of blog posts on EfficientDet](https://amaarora.github.io/2021/01/11/efficientdet.html#bifpn-bi-directional-feature-pyramid-network)
2.   [Arictle: EfficientDet: Guide to State of The Art Object Detection Model
](https://analyticsindiamag.com/efficientdet/)





